In [1]:
import torchshow as ts
from data.dali.module import DALIDataModule
from tqdm import tqdm
import torch
import sys
import numpy as np
import pickle

# sys.path.append('/home/whx/SRFormer')

In [2]:
VAR_LIST = ['msl', '2t', '10u', '10v', 
            'hgtn_5000', 'hgtn_10000', 'hgtn_15000', 'hgtn_20000', 'hgtn_25000', 'hgtn_30000', 'hgtn_40000', 'hgtn_50000', 'hgtn_60000', 'hgtn_70000', 'hgtn_85000', 'hgtn_92500', 'hgtn_100000', 
            'u_5000', 'u_10000', 'u_15000', 'u_20000', 'u_25000', 'u_30000', 'u_40000', 'u_50000', 'u_60000', 'u_70000', 'u_85000', 'u_92500', 'u_100000', 
            'v_5000', 'v_10000', 'v_15000', 'v_20000', 'v_25000', 'v_30000', 'v_40000', 'v_50000', 'v_60000', 'v_70000', 'v_85000', 'v_92500', 'v_100000', 
            't_5000', 't_10000', 't_15000', 't_20000', 't_25000', 't_30000', 't_40000', 't_50000', 't_60000', 't_70000', 't_85000', 't_92500', 't_100000', 
            'q_5000', 'q_10000', 'q_15000', 'q_20000', 'q_25000', 'q_30000', 'q_40000', 'q_50000', 'q_60000', 'q_70000', 'q_85000', 'q_92500', 'q_100000'
]

In [23]:
position_file = 'configs/datasets/goes_hw64x64.txt'
data_dir = {
        "hrrr": 'hrrr/hourly2_fixed_TMP_L103',
        "era5": 'era5_us_npy/npy',
    }
input_len, output_len = 1, 1
time_config = {
        'hrrr': {'input': input_len, 'output': output_len, 'gap': 1}, 
        'era5': {'input': input_len, 'output': output_len, 'gap': 1}, 
    }
crop_h= 32
crop_w= 32
crop_cnt= 1
batch_size= 4
pad= 0
margin= 0
h= 105
w= 179
mount_paths = ['/blob/kmsw0westus3/kms1']

In [24]:
data_module = DALIDataModule(data_dir=data_dir, batch_size=batch_size, num_threads=8, 
                            time_config=time_config, mount_paths=mount_paths, 
                            crop_cnt=crop_cnt, crop_h=crop_h, crop_w=crop_w, h=h, w=w, 
                            margin=margin, pad=pad, train_end_datetime='2020123100')

data_module.prepare()
data_module.setup()
dataloader = data_module.val_dataloader()

t_dict = {}
for var in VAR_LIST:
    t_dict[var] = {}
    for t in range(input_len + output_len):
        t_dict[var][t] = {'mean': [], 'std': []}
    

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacty of 44.55 GiB of which 297.56 MiB is free. Process 2957527 has 12.26 GiB memory in use. Including non-PyTorch memory, this process has 31.93 GiB memory in use. Of the allocated memory 19.79 GiB is allocated by PyTorch, and 19.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
for batch in tqdm(dataloader):
    print(batch['input_hrrr']['surface'].shape)

    print(batch['input_era5']['surface'].shape)
    # atmo_hrrr = batch['input_hrrr']['pressure'].reshape(-1, 1, 65, 1050, 1790).squeeze(1)
    # data_hrrr = torch.cat([surf_hrrr, atmo_hrrr], dim=1)
    # print(data_hrrr.shape)

    # surf_era5 = batch['input_era5']['surface'].reshape(-1, 1, 4, 105, 179).squeeze(1)
    # atmo_era5 = batch['input_era5']['pressure'].reshape(-1, 1, 65, 105, 179).squeeze(1)
    # data_era5 = torch.cat([surf_era5, atmo_era5], dim=1)
    # print(data_era5.shape)

    break

  0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: shape '[4, 1, -1, 40, 40]' is invalid for input of size 266240

### New-0220

In [ ]:
import pandas as pd
import os   
import numpy as np
from datetime import datetime, timedelta
from copy import deepcopy
from tqdm import tqdm
import torch

variable_name=['msl', '2t', '10u', '10v', 
'hgtn_5000', 'hgtn_10000', 'hgtn_15000', 'hgtn_20000', 'hgtn_25000','hgtn_30000', 'hgtn_40000', 'hgtn_50000', 'hgtn_60000','hgtn_70000', 'hgtn_85000', 'hgtn_92500', 'hgtn_100000',
'u_5000', 'u_10000', 'u_15000', 'u_20000', 'u_25000', 'u_30000', 'u_40000', 'u_50000', 'u_60000', 'u_70000', 'u_85000', 'u_92500','u_100000',
'v_5000', 'v_10000', 'v_15000','v_20000','v_25000', 'v_30000','v_40000', 'v_50000', 'v_60000','v_70000','v_85000','v_92500','v_100000',
't_5000','t_10000','t_15000','t_20000','t_25000','t_30000','t_40000', 't_50000', 't_60000','t_70000','t_85000','t_92500','t_100000',
'q_5000','q_10000','q_15000','q_20000','q_25000','q_30000','q_40000','q_50000','q_60000','q_70000','q_85000','q_92500','q_100000',]

ALL_STDS = torch.Tensor([6.8789e+02, 1.1408e+01, 3.4087e+00, 3.8128e+00, 1.9669e+02, 2.2751e+02,
2.8561e+02, 3.0650e+02, 2.9603e+02, 2.7141e+02, 2.1778e+02, 1.7208e+02,
1.3423e+02, 1.0113e+02, 6.2827e+01, 5.5131e+01, 5.6151e+01, 9.4211e+00,
1.0859e+01, 1.3950e+01, 1.7115e+01, 1.7562e+01, 1.6353e+01, 1.3276e+01,
1.0801e+01, 8.9426e+00, 7.5022e+00, 6.1020e+00, 5.4538e+00, 3.7264e+00,
4.3335e+00, 7.7346e+00, 1.2055e+01, 1.6379e+01, 1.7496e+01, 1.6438e+01,
1.3244e+01, 1.0723e+01, 8.8519e+00, 7.4927e+00, 6.6357e+00, 6.3794e+00,
4.4868e+00, 4.5961e+00, 6.9949e+00, 5.8210e+00, 4.5001e+00, 4.9734e+00,
6.2033e+00, 7.2265e+00, 7.4370e+00, 7.7510e+00, 8.4697e+00, 9.9353e+00,
1.0570e+01, 1.0831e+01, 1.0117e-06, 2.1395e-06, 4.7380e-06, 1.8255e-05,
6.3045e-05, 1.5294e-04, 4.8075e-04, 9.9271e-04, 1.6142e-03, 2.2498e-03,
3.5426e-03, 4.3892e-03, 5.0458e-03])

ALL_MEANS = torch.Tensor([1.0154e+05,  2.8764e+02,  3.6334e-01, -2.1942e-01,  2.0653e+04,
1.6401e+04,  1.3901e+04,  1.2086e+04,  1.0639e+04,  9.4158e+03,
7.3890e+03,  5.7308e+03,  4.3219e+03,  3.0932e+03,  1.4998e+03,
7.9164e+02,  1.2938e+02,  1.9525e+00,  1.3733e+01,  2.1062e+01,
2.3357e+01,  2.1682e+01,  1.9152e+01,  1.4822e+01,  1.1510e+01,
8.7861e+00,  6.3429e+00,  2.7486e+00,  1.2042e+00,  3.4059e-01,
-1.9104e-01,  1.5638e-01,  3.5123e-01,  4.5588e-01,  4.4878e-01,
3.9698e-01,  2.3847e-01,  1.6056e-01,  1.3170e-01,  1.8216e-01,
3.9488e-01,  3.1365e-01, -2.5557e-01,  2.1192e+02,  2.0901e+02,
2.1325e+02,  2.1837e+02,  2.2547e+02,  2.3356e+02,  2.4815e+02,
2.5946e+02,  2.6831e+02,  2.7558e+02,  2.8359e+02,  2.8666e+02,
2.9031e+02,  2.8537e-06,  3.0846e-06,  6.6055e-06,  2.4169e-05,
7.3131e-05,  1.6582e-04,  4.9159e-04,  1.0437e-03,  1.8908e-03,
2.9435e-03,  5.3577e-03,  7.2076e-03,  8.8479e-03])

In [23]:
# from datetime import datetime, timedelta
# import numpy as np
# import pandas as pd
# import os 
# from datetime import datetime, timedelta
# from copy import deepcopy
# from tqdm import tqdm
# import torch
# from torch.utils.data import Dataset, DataLoader

# class HRRRDataset(Dataset):
#     def __init__(self, file_names, root_dir, means, stds, device='cpu'):
#         """
#         Args:
#             file_names (list of str): List of file names to be loaded.
#             root_dir (str): Directory with all the .npy files.
#             means (torch.Tensor): Tensor of means for normalization.
#             stds (torch.Tensor): Tensor of standard deviations for normalization.
#             device (str): Device to load the data onto, default is 'cpu'.
#         """
#         self.file_names = file_names
#         self.root_dir = root_dir
#         self.means = means.to(device)
#         self.stds = stds.to(device)
#         self.device = device

#     def __len__(self):
#         return len(self.file_names)

#     def __getitem__(self, idx):
#         file_path = os.path.join(self.root_dir, self.file_names[idx])
#         data = np.load(file_path)
#         data = torch.from_numpy(data).float().to(self.device)
#         start_dim3 = (data.size(2) - 1050) // 2  
#         end_dim3 = start_dim3 + 1050  
#         start_dim4 = (data.size(3) - 1790) // 2  
#         end_dim4 = start_dim4 + 1790  
#         data = data[:, :, start_dim3:end_dim3, start_dim4:end_dim4]
#         data = (data - self.means[:, None, None]) / self.stds[:, None, None]
#         return data

# def generate_file_names(start_date, end_date):
#     """
#     Generates a list of file names for each hour between the start and end dates.
#     """
#     file_names = []
#     current_date = start_date
#     while current_date <= end_date:
#         for hour in range(24):
#             file_names.append(current_date.strftime('%Y%m%d') + f'{hour:02d}.npy')
#         current_date += timedelta(days=1)
#     return file_names

# # Set random seed for reproducibility
# seed = 42
# torch.manual_seed(seed)
# np.random.seed(seed)

# # Define date range
# start_date = datetime(2019, 1, 1)
# end_date = datetime(2020, 12, 31)

# # Generate file names
# file_names = generate_file_names(start_date, end_date)

# # Shuffle and split file names into training, testing, and validation sets
# np.random.shuffle(file_names)
# total_files = len(file_names)
# train_size = int(total_files * 0.8)
# test_size = int(total_files * 0.1)
# val_size = total_files - train_size - test_size

# train_files = file_names[:train_size]
# test_files = file_names[train_size:train_size + test_size]
# val_files = file_names[train_size + test_size:]


# # Directory containing .npy files
# hrrr_root = "/blob/kmsw0westus3/kms1/hrrr/hourly2_fixed_TMP_L103"  
# device = 'cuda:1'

# # Create dataset instances
# train_dataset = HRRRDataset(train_files, hrrr_root, ALL_MEANS, ALL_STDS,device)
# test_dataset = HRRRDataset(test_files, hrrr_root, ALL_MEANS, ALL_STDS,device)
# val_dataset = HRRRDataset(val_files, hrrr_root, ALL_MEANS, ALL_STDS,device)

# # Create DataLoader instances
# batch_size = 64  # Adjust based on your system's capability
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)




### 将hrrr数据存成h5

In [ ]:
# ! pip install h5py
# import h5py
# import numpy as np
# import os
# from datetime import datetime, timedelta

# def preprocess_data(data):
#     # 这里添加你的数据预处理代码，比如裁剪、规范化等
#     start_dim3 = (data.shape[2] - 1050) // 2  
#     end_dim3 = start_dim3 + 1050  
#     start_dim4 = (data.shape[3] - 1790) // 2  
#     end_dim4 = start_dim4 + 1790  
#     data = data[:, :, start_dim3:end_dim3, start_dim4:end_dim4]
#     return data

# def save_to_hdf5(file_names, root_dir, output_file):
#     with h5py.File(output_file, 'w') as h5f:
#         for i, file_name in enumerate(file_names):
#             file_path = os.path.join(root_dir, file_name)
#             # 加载原始数据
#             data = np.load(file_path)
#             # 预处理数据
#             data = preprocess_data(data)
#             # 将数据存储到 HDF5 文件中
#             h5f.create_dataset(name=file_name, data=data, compression="gzip")
#             print(f'Processed and saved {file_name} to {output_file}')

# def generate_file_names(start_date, end_date):
#     file_names = []
#     current_date = start_date
#     while current_date <= end_date:
#         for hour in range(24):
#             file_names.append(current_date.strftime('%Y%m%d') + f'{hour:02d}.npy')
#         current_date += timedelta(days=1)
#     return file_names

# # 定义日期范围
# start_date = datetime(2019, 1, 1)
# end_date = datetime(2020, 12, 31)

# # 生成文件名列表
# file_names = generate_file_names(start_date, end_date)

# # 定义数据根目录和输出文件路径
# hrrr_root = "/blob/kmsw0westus3/kms1/hrrr/hourly2_fixed_TMP_L103"  # 更新为你的.npy文件的路径
# output_h5_file = "/home/whx/data/hrrr_hourly2_fixed_TMP_L103.h5"  # 输出HDF5文件的路径

# # 保存数据到 HDF5
# save_to_hdf5(file_names, hrrr_root, output_h5_file)


In [ ]:
hrrr_root = "/blob/kmsw0westus3/kms1/hrrr/hourly2_fixed_TMP_L103"
# for file in os.listdir(hrrr_root):
#     print(file)
# 2019-01-01 ~ 2020-12-31 每天24个npy

year, month, day_start, day_end = 2019, 1, 1, 2 # 2020, 8, 9, 12 # 2019, 10, 26, 29
# case_timestamp = f"{year}_{month}_{day_start}_{day_end}"
device = 'cuda:1'
start_time = datetime(year=year, month=month, day=day_start)
end_time = datetime(year=year, month=month, day=day_end)
dts = []
while start_time < end_time:
    dts.append(deepcopy(start_time))
    start_time += timedelta(hours=1)
data = []
for d in tqdm(dts):
    data.append(np.load(os.path.join(hrrr_root, d.strftime('%Y%m%d%H.npy'))))
data = np.concatenate(data, axis=0)
case_data = torch.from_numpy(data[:, :, :, :]).to(device)
all_means = ALL_MEANS.to(device)
all_stds = ALL_STDS.to(device)
case_data = ((case_data - all_means[None,:,None,None]) / all_stds[None,:,None,None])    

In [ ]:
case_data.shape

In [10]:
import os   
import numpy as np
from datetime import datetime, timedelta
from copy import deepcopy
from tqdm import tqdm
import torch

era5_root = "/blob/kmsw0westus3/kms1/era5_us_npy/npy/"
# for file in os.listdir(era5_root):
#     print(file)
# 2019-01-01 ~ 2020-12-31 每天24个npy
year, month, day_start, day_end = 2019, 1, 1, 2 # 2020, 8, 9, 12 # 2019, 10, 26, 29
# case_timestamp = f"{year}_{month}_{day_start}_{day_end}"
device = 'cuda:1'
start_time = datetime(year=year, month=month, day=day_start)
end_time = datetime(year=year, month=month, day=day_end)
dts = []
while start_time < end_time:
    dts.append(deepcopy(start_time))
    start_time += timedelta(hours=1)
data = []
for d in tqdm(dts):
    data.append(np.load(os.path.join(era5_root, d.strftime('%Y%m%d%H.npy'))))
data = np.concatenate(data, axis=0)
era5_case_data = torch.from_numpy(data[:, :, :, :]).to(device)
era5_case_data = ((era5_case_data - all_means[None,:,None,None]) / all_stds[None,:,None,None])   

100%|██████████| 24/24 [00:10<00:00,  2.28it/s]


NameError: name 'all_means' is not defined

In [ ]:
era5_case_data.shape